# Athena Iceberg - Data Eng Use Cases

## Initial testing

In [1]:
import awswrangler as wr
import pydbtools as pydb
import py_aws_vault_auth
import os

In [2]:
comparison = "athena_iceberg"
region = "eu-west-1"
bucketname = "sb-test-bucket-ireland"
db_name = "wto_hudi_iceberg"
s3_root_folder = "wo/de_use_cases"
s3_base_path = f"s3://{bucketname}/{s3_root_folder}/{comparison}"
db_base_path = f"{s3_base_path}database/"

environ_auth = py_aws_vault_auth.authenticate("sso-sandbox", prompt="python", return_as="environ")
os.environ.update(environ_auth)

### Bulk insert and add curation columns 

Set up variables for bulk insert test

In [3]:
senario = "bulk_insert"
source_fl = f"s3://sb-test-bucket-ireland/dummy_data/full_load/"
source_ud = f"s3://sb-test-bucket-ireland/dummy_data/updates/"
temp_table_name = f"{comparison}_{senario}_temp"
dest_table_name = f"{comparison}_{senario}_iceberg"

Create a temporary table from source

In [4]:
temp_table_sql_1 = f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.{temp_table_name} (
        product_id string,
        product_name string,
        price int,
        extraction_timestamp timestamp,
        op string
    )
    STORED AS PARQUET
    LOCATION '{source_fl}'
"""
#wr.athena.read_sql_query(sql=temp_table_sql, database=db_name, ctas_approach=False)
# wr.athena.read_sql_query(f"DROP TABLE {temp_table_name}", database=db_name, ctas_approach=False)


Check that the table is populated

In [ ]:
wr.athena.read_sql_query(f"SELECT * FROM {temp_table_name}", database=db_name, ctas_approach=False)

Create an iceberg table from source table

In [ ]:
dest_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {db_name}.{dest_table_name}
        WITH (table_type='ICEBERG',
        location='{db_base_path}{senario}/',
        format='PARQUET',
        is_external=false)
        AS SELECT
            product_id,
            product_name,
            price,
            CAST(extraction_timestamp AS timestamp(6)) AS extraction_timestamp,
            op 
           FROM {db_name}.{temp_table_name};
"""
wr.athena.read_sql_query(sql=dest_table_sql, database=db_name, ctas_approach=False, workgroup='Athena3')
wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name}", database=db_name, ctas_approach=False, workgroup='Athena3')
##wr.athena.delete_table(database=db_name, table=temp_table_name)

## 13 sec

Update destination iceberg table with new columns

In [ ]:
add_columns_sql = f"""
    ALTER TABLE {db_name}.{dest_table_name}
    ADD COLUMNS (start_datetime TIMESTAMP, end_datetime TIMESTAMP, is_current BOOLEAN)
"""
update_values_sql = f"""
    UPDATE {db_name}.{dest_table_name}
    SET start_datetime = extraction_timestamp, 
        end_datetime = CAST(TIMESTAMP '2250-01-01' as TIMESTAMP(6)), 
        is_current = true
"""
#wr.athena.read_sql_query(sql=update_values_sql, database=db_name, ctas_approach=False, workgroup='Athena3')
print("Updated values")
wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name}", database=db_name, ctas_approach=False, workgroup='Athena3')
##wr.athena.delete_table(database=db_name, table=temp_table_name)

Delete and recreate tempory table from update file

In [ ]:
wr.athena.read_sql_query(f"DROP TABLE IF EXISTS {temp_table_name}", database=db_name, ctas_approach=False)
temp_table_sql = f"""
    CREATE EXTERNAL TABLE {db_name}.{temp_table_name} (
        product_id string,
        product_name string,
        price int,
        extraction_timestamp timestamp,
        op string
    )
    STORED AS PARQUET
    LOCATION '{source_ud}'
"""
wr.athena.read_sql_query(sql=temp_table_sql, database=db_name, ctas_approach=False)
wr.athena.read_sql_query(f"SELECT * FROM {temp_table_name}", database=db_name, ctas_approach=False)


Update destination table when key is source (CDC / update) table

In [ ]:
update_dest_sql = f"""
    MERGE INTO {db_name}.{dest_table_name} dest
        USING {db_name}.{temp_table_name} sour
            ON sour.product_id = dest.product_id
    WHEN MATCHED AND dest.is_current = TRUE AND sour.extraction_timestamp > dest.extraction_timestamp
        THEN UPDATE
            SET end_datetime = sour.extraction_timestamp, is_current = FALSE;
"""
wr.athena.read_sql_query(sql=update_dest_sql, database=db_name, ctas_approach=False)
wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name}", database=db_name, ctas_approach=False)

Insert all updates from source table

In [ ]:
insert_dest_sql = f"""
INSERT INTO {db_name}.{dest_table_name}
    SELECT product_id, product_name, price, CAST(extraction_timestamp AS TIMESTAMP(6)), op, 
      CAST(extraction_timestamp AS TIMESTAMP(6)), CAST(TIMESTAMP '2250-01-01' as TIMESTAMP(6)),TRUE
    FROM {db_name}.{temp_table_name}
"""
wr.athena.read_sql_query(sql=update_dest_sql, database=db_name, ctas_approach=False)
wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name}", database=db_name, ctas_approach=False)

In [ ]:
wr.athena.read_sql_query(sql=temp_table_sql, database=db_name, ctas_approach=False)
wr.athena.read_sql_query(f"DROP TABLE {temp_table_name}", database=db_name, ctas_approach=False)


### Summary of changes

**SETUP**
1. Create a temp table from FL
2. Create iceberg full load via CTAS, adding mojap fields
3. Create CDC temp table 
4. Create a CDC view adding mojap fields (didnt actualy do this last light as plain insert was quick enough for Sou's critera)

**PROCESSING**
1. Use merge to close is_current records in iceberg that exist in CDC (there is an issue of closing date flif multiple CDC)
2. Insert CDC into iceberg

**NEXT STEPS**
1. Run processing as a single step
2. Update the cdc insrt to a view
3. Close multiple CDC updates with previous date



## Functional Testing

### General Setup

Imports

SSO Variables. Run below in terminal:

```
# pip3 install python-dotenv
aws-vault exec sso-sandbox
source /Users/william.orr/Developer/Projects/hudi-vs-iceberg/venv/bin/activate
env | grep AWS > .env
```

In [1]:
import athena_functions as af
import awswrangler as wr
import datetime
from dotenv import load_dotenv
from importlib import reload
import os

load_dotenv()

True

In [2]:
input_data_directory = (
    "s3://sb-test-bucket-ireland/data-engineering-use-cases/dummy-data/"
)
## Assumptions about real setup
## data-engineering-use-cases/realistic-data/<column_count>_<row_count>/<test_type>/<name>00<x>.parquet
## .../realistic-data/10_10K/full_load/
## .../realistic-data/10_100M/updates/
## .../realistic-data/100_10B/late_updates/

full_load_filepath = f"{input_data_directory}full_load/full_load.parquet"
updates_filepath = f"{input_data_directory}updates/updates.parquet"
late_updates_filepath = f"{input_data_directory}late_updates/late_updates.parquet"
output_data_directory = (
    "s3://sb-test-bucket-ireland/wo/de-usecases/athena/"
)

db_name = "wto_hudi_iceberg"
real_test_cases = ["10_1K", "10_1M", "10_1B", "10_10B", "100_1K", "100_1M", "100_1B", "100_10B"]
current_test = full_load_filepath.split("/")[-3].replace("-", "_")
dest_table_name = f"ICE_{current_test}"
future_end_datetime = datetime.datetime(2250, 1, 1)
primary_key = "product_id"

### Athena Specific Setup

#### Create source full load & update tables
(This will be done in glue for realistic data)

In [4]:
## Create source table name in format <test>_full_load, <test>_updates
for table in ["full_load", "updates", "late_updates"]:
    
    source_folder = f"{input_data_directory}{table}/"
    create_table_query = f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS {db_name}.dummy_data_{table} (
        product_id string,
        product_name string,
        price int,
        extraction_timestamp TIMESTAMP,
        op string
    )
    STORED AS PARQUET
    LOCATION '{source_folder}'
    """
    query_id = wr.athena.start_query_execution(
            sql=create_table_query, database=db_name, workgroup='Athena3') #, workgroup='Athena3'
    print(f"Creating table {table} in {db_name} database with query ID {query_id}\n")

Creating table full_load in wto_hudi_iceberg database with query ID 089b03c1-1f44-4859-ad30-1582bf1183f3

Creating table updates in wto_hudi_iceberg database with query ID 20e889f7-95cc-4400-8fe4-84f266f93759

Creating table late_updates in wto_hudi_iceberg database with query ID 09df0458-ad31-4c56-9933-4ec80a271a16



#### 1. Bulk Insert

Generate SQL for live data in Glue

In [5]:
bulk_insert_filepath = af.bulk_insert(
    full_load_filepath, output_data_directory, future_end_datetime, demo="print_sql")
print(bulk_insert_filepath)


        **DROP DEST TABLE** 
    
    DROP TABLE IF EXISTS wto_hudi_iceberg.ICE_dummy_data;

        **CREATE DEST TABLE** 
    
    CREATE TABLE IF NOT EXISTS wto_hudi_iceberg.ICE_dummy_data
        WITH (table_type='ICEBERG',
        location='s3://sb-test-bucket-ireland/wo/de-usecases/athena/database/ICE_dummy_data',
        format='PARQUET',
        is_external=false)
        AS SELECT
            src.product_id, src.product_name, src.price, 
            CAST(extraction_timestamp AS TIMESTAMP(6)) AS extraction_timestamp, src.op,
            CAST(extraction_timestamp AS TIMESTAMP(6)) AS start_datetime,
            CAST(TIMESTAMP '2250-01-01 00:00:00' AS TIMESTAMP(6)) AS end_datetime,
            CAST(true AS boolean) AS is_current
           FROM wto_hudi_iceberg.dummy_data_full_load src;

        


For demo data insert data to iceberg table (drop existing)


In [3]:
bulk_insert_filepath = af.bulk_insert(
    full_load_filepath, output_data_directory, future_end_datetime, demo="demo")

wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name}", database=db_name, ctas_approach=False, workgroup='Athena3')

completed DROP in 1710 milliseconds,  0 bytes scanned
completed INSERT in 2992 milliseconds,  490 bytes scanned
total MB scanned: 0.0004673004150390625 in 4.702 seconds, ~ Cost =  $2.228262019343674e-09


,product_id,product_name,price,extraction_timestamp,op,start_datetime,end_datetime,is_current
0,00001,Heater,250,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2250-01-01,True
1,00002,Thermostat,400,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2250-01-01,True
2,00003,Television,600,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2250-01-01,True
3,00004,Blender,100,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2250-01-01,True
4,00005,USB charger,50,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2250-01-01,True


#### Maintainence Scripts

Reload module

In [7]:
reload(af)

<module 'athena_functions' from '/Users/william.orr/Developer/Projects/hudi-vs-iceberg/hudi_vs_iceberg/athena_iceberg/athena_functions.py'>

Drop destination ICE table

In [45]:
wr.athena.read_sql_query(f"DROP TABLE IF EXISTS {dest_table_name}", database=db_name, ctas_approach=False, workgroup='Athena3')

""


Testing complex merge

In [28]:
### TESTING
sql1 = """INSERT INTO wto_hudi_iceberg.ICE_dummy_data
        SELECT src.product_id, src.product_name, src.price, 
            CAST(src.extraction_timestamp AS TIMESTAMP(6)) AS extraction_timestamp, src.op,
            CAST(src.extraction_timestamp AS TIMESTAMP(6)), NULL, NULL
        FROM wto_hudi_iceberg.dummy_data_late_updates src"""
sql4 = "DROP TABLE IF EXISTS wto_hudi_iceberg.ICE_dummy_data_temp"
sql2 = """    CREATE TABLE IF NOT EXISTS wto_hudi_iceberg.ICE_dummy_data_temp
        WITH (table_type='ICEBERG',
        location='s3://sb-test-bucket-ireland/wo/de-usecases/athena/database/ICE_dummy_data_temp',
        format='PARQUET',
        is_external=false)
    AS 
    WITH end_date AS (
        SELECT product_id, extraction_timestamp, LEAD(extraction_timestamp, 1, TIMESTAMP '2250-01-01 00:00:00')
            OVER (PARTITION BY product_id 
        ORDER BY extraction_timestamp) AS end_datetime_lead
        FROM wto_hudi_iceberg.ICE_dummy_data
    )
    SELECT product_id, extraction_timestamp, end_datetime_lead,
        CASE WHEN end_datetime_lead = CAST(TIMESTAMP '2250-01-01 00:00:00' AS TIMESTAMP(6)) THEN true
            ELSE false END AS is_current 
    FROM end_date
"""
sql2_5 = """ SELECT ice.*, tmp.*
    FROM wto_hudi_iceberg.ICE_dummy_data ice
    JOIN wto_hudi_iceberg.ICE_dummy_data_temp tmp
    ON (tmp.product_id = ice.product_id
    AND tmp.extraction_timestamp = ice.extraction_timestamp)"""
sql3 = """MERGE INTO wto_hudi_iceberg.ICE_dummy_data ice
    USING wto_hudi_iceberg.ICE_dummy_data_temp tmp
    ON (tmp.product_id = ice.product_id
    AND tmp.extraction_timestamp = ice.extraction_timestamp)
    WHEN MATCHED 
        THEN UPDATE
            SET ice.end_datetime = tmp.end_datetime_lead,
                ice.is_current = tmp.is_current"""
resp = wr.athena.start_query_execution(
    sql=sql3, database="wto_hudi_iceberg", workgroup='Athena3', wait=True
)
resp

InvalidRequestException: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 7:20: mismatched input '.'. Expecting: '='

#### 2. Simple CDC

For live data return insert sql script

In [4]:
scd2_simple_sql_queries = af.scd2_simple(
    ##NOTE:pandas example uses "bulk_insert_filepath" generated from previous query
    full_load_filepath,
    updates_filepath,
    output_data_directory,
    future_end_datetime,
    primary_key,
    demo="print_sql"
)
print(scd2_simple_sql_queries)

**SIMPLE INSERT**

    INSERT INTO wto_hudi_iceberg.ICE_dummy_data
        SELECT src.product_id, src.product_name, src.price, 
            CAST(src.extraction_timestamp AS TIMESTAMP(6)) AS extraction_timestamp, src.op,
            CAST(src.extraction_timestamp AS TIMESTAMP(6)), NULL, NULL
        FROM wto_hudi_iceberg.dummy_data_updates src


**SIMPLE MERGE**

    MERGE INTO wto_hudi_iceberg.ICE_dummy_data dest
    USING wto_hudi_iceberg.dummy_data_updates src
        ON src.product_id = dest.product_id
    WHEN MATCHED AND dest.is_current = TRUE 
        THEN UPDATE
            SET end_datetime = src.extraction_timestamp, is_current = FALSE
    WHEN MATCHED AND dest.extraction_timestamp = dest.start_datetime
        THEN UPDATE
        SET end_datetime = CAST(TIMESTAMP '2250-01-01 00:00:00' AS TIMESTAMP(6))
        , is_current = TRUE;



For demo data update demo iceberg table

In [4]:
scd2_simple_filepath = af.scd2_simple(
    full_load_filepath, 
    updates_filepath,
    output_data_directory, 
    future_end_datetime, 
    primary_key)
wr.athena.read_sql_query(f"SELECT * FROM {dest_table_name} ORDER BY {primary_key}, extraction_timestamp", database=db_name, ctas_approach=False, workgroup='Athena3')

completed INSERT in 2472 milliseconds,  481 bytes scanned
completed MERGE in 2472 milliseconds,  481 bytes scanned
total MB scanned: 0.0009174346923828125 in 4.944 seconds, ~ Cost =  $4.374669515527785e-09


,product_id,product_name,price,extraction_timestamp,op,start_datetime,end_datetime,is_current
0,00001,Heater,250,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2023-01-01 01:01:01,False
1,00001,Heater,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
2,00002,Thermostat,400,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2023-01-01 01:01:01,False
3,00002,Thermostat,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
4,00003,Television,600,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2023-01-01 01:01:01,False
5,00003,Television,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
6,00004,Blender,100,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2023-01-01 01:01:01,False
7,00004,Blender,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
8,00005,USB charger,50,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2023-01-01 01:01:01,False
9,00005,USB charger,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True


#### 3. Complex CDC

For live data return sql script

In [27]:
scd2_complex = af.scd2_complex(
    ##NOTE:pandas example uses "bulk_insert_filepath" generated from previous query
    full_load_filepath,
    late_updates_filepath,
    output_data_directory,
    future_end_datetime,
    primary_key,
    demo="print_sql"
)
print(scd2_complex)


        **SIMPLE INSERT**

    INSERT INTO wto_hudi_iceberg.ICE_dummy_data
        SELECT src.product_id, src.product_name, src.price, 
            CAST(src.extraction_timestamp AS TIMESTAMP(6)) AS extraction_timestamp, src.op,
            CAST(src.extraction_timestamp AS TIMESTAMP(6)), NULL, NULL
        FROM wto_hudi_iceberg.dummy_data_late_updates src

        **COMPLEX TEMP TABLE**

    CREATE TABLE IF NOT EXISTS wto_hudi_iceberg.ICE_dummy_data_temp
        WITH (table_type='ICEBERG',
        location='s3://sb-test-bucket-ireland/wo/de-usecases/athena/database/ICE_dummy_data_temp',
        format='PARQUET',
        is_external=false)
    AS 
    WITH end_date AS (
        SELECT product_id, extraction_timestamp, LEAD(extraction_timestamp, 1, TIMESTAMP '2250-01-01 00:00:00')
            OVER (PARTITION BY product_id 
        ORDER BY extraction_timestamp) AS end_datetime_lead
        FROM wto_hudi_iceberg.ICE_dummy_data
    )
    SELECT product_id, extraction_timestamp, end_datetim

For demo data run insert & merge to iceberg table

In [5]:
scd2_complex = af.scd2_complex(
    ##NOTE:pandas example uses "bulk_insert_filepath" generated from previous query
    full_load_filepath,
    late_updates_filepath,
    output_data_directory,
    future_end_datetime,
    primary_key,
    demo="demo"
)
print(scd2_complex)
wr.athena.read_sql_query("SELECT * FROM wto_hudi_iceberg.ICE_dummy_data", database=db_name, ctas_approach=False, workgroup='Athena3')

INSERT CDC's ...
Completed INSERT in 2793 milliseconds, scanned 481 bytes
        CREATE TEMP TABLE ...
Completed CREATE TEMP TABLE in 3701 milliseconds, scanned 306 bytes
        MERGE UPDATES ...
Completed MERGE in 6502 milliseconds, scanned 1311 bytes
        DROP TEMP TABLE ...
Completed DROP TEMP TABLE in 1132 milliseconds, scanned 0 bytes
Total Scanned MB: 0.0020008087158203125 in 14.128 seconds, ~ Cost = $9.540599421598017e-09
s3://sb-test-bucket-ireland/wo/de-usecases/athena/database/ICE_dummy_data


,product_id,product_name,price,extraction_timestamp,op,start_datetime,end_datetime,is_current
0,00002,Thermostat,400,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2022-06-01 01:01:01,False
1,00003,Television,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
2,00004,Blender,100,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2022-06-01 01:01:01,False
3,00003,Television,600,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2022-06-01 01:01:01,False
4,00004,Blender,500,2022-06-01 01:01:01,U,2022-06-01 01:01:01,2023-01-01 01:01:01,False
5,00004,Blender,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
6,00005,USB charger,50,2022-01-01 01:01:01,<NA>,2022-01-01 01:01:01,2022-06-01 01:01:01,False
7,00001,Heater,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True
8,00001,Heater,500,2022-06-01 01:01:01,U,2022-06-01 01:01:01,2023-01-01 01:01:01,False
9,00005,USB charger,1000,2023-01-01 01:01:01,U,2023-01-01 01:01:01,2250-01-01 00:00:00,True


In [14]:
## Cost comparision
glue_total_time_sec = 0.697 + 0.327 + 0.579
glue_cost_per_hr = 0.44
glue_total_cost = glue_total_time_sec * glue_cost_per_hr/60**2
print(f"~ Total Glue Cost = ${glue_total_cost}")

athena_total_time_sec = 4.702 + 4.944  + 14.128 
athena_total_data_mb = 0.0004673004150390625 + 0.0009174346923828125 + 0.0020008087158203125
athena_cost_per_tb = 5.00
athena_total_cost = athena_total_data_mb * athena_cost_per_tb/1024**2
print(f"~ Total Athena Cost = ${athena_total_cost}")
print(f"pandas on glue was {glue_total_cost/athena_total_cost:.0f} times more expensive than athena, but {athena_total_time_sec/glue_total_time_sec:.0f} times faster")


~ Total Glue Cost = $0.00019592222222222222
~ Total Athena Cost = $1.6143530956469476e-08
pandas on glue was 12136 times more expensive than athena, but 15 times faster
